In [35]:
import requests
import pandas as pd
import numpy as np
import keys
import json
from tqdm import tqdm

In [3]:
states = list(pd.read_csv('states.csv').Abbreviation)

In [68]:
frame_list = []
success = []
fail = []
for state in tqdm(states):
    for page in range(1,30):
        try:
            url ='https://api.seatgeek.com/2/events?venue.state='
            req = requests.get(f'{url}{state}&page={page}&type=concert&client_id={keys.seatgeek}')
            x = json.loads(req.text)
            df = pd.DataFrame(x['events'])
            frame_list.append(df)
            success.append(f'{state},{page}')
        except:
            fail.append(f'{state},{page}')

100%|██████████| 51/51 [03:45<00:00,  3.75s/it]


In [71]:
seatgeek = pd.concat(frame_list)

/Users/matt/.pyenv/versions/anaconda3-2019.03/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [80]:
seatgeek.reset_index(inplace=True)

In [81]:
seatgeek.columns

Index(['index', 'access_method', 'announce_date', 'announcements',
       'conditional', 'created_at', 'date_tbd', 'datetime_local',
       'datetime_tbd', 'datetime_utc', 'description', 'enddatetime_utc',
       'event_promotion', 'general_admission', 'id', 'is_open', 'links',
       'performers', 'popularity', 'score', 'short_title', 'stats', 'status',
       'taxonomies', 'time_tbd', 'title', 'type', 'url', 'venue',
       'visible_until_utc'],
      dtype='object')

In [103]:
seatgeek.drop(columns = ['index', 'announce_date','access_method','description','taxonomies','announcements', 'conditional', 'date_tbd','datetime_tbd', 'enddatetime_utc',
                         'event_promotion','performers','type','time_tbd','links','status','is_open','datetime_utc', 'visible_until_utc','title'],inplace=True)

In [116]:
seatgeek.stats[2164]

{'visible_listing_count': 2,
 'dq_bucket_counts': [0, 0, 0, 0, 0, 0, 0, 0],
 'average_price': 83,
 'lowest_sg_base_price_good_deals': None,
 'lowest_price_good_deals': None,
 'median_price': 83,
 'listing_count': 8,
 'variants': None,
 'lowest_price': 45,
 'lowest_sg_base_price': 32,
 'highest_price': 136}

In [121]:
seatgeek['listing_count']=seatgeek.stats.apply(lambda x: x['listing_count'])
seatgeek['average_price']=seatgeek.stats.apply(lambda x: x['average_price'])
seatgeek['median_price']=seatgeek.stats.apply(lambda x: x['median_price'])
seatgeek['lowest_price']=seatgeek.stats.apply(lambda x: x['lowest_price'])
seatgeek.drop(columns=['stats'], inplace =True)

In [123]:
seatgeek.venue[0]

{'links': [],
 'metro_code': 686,
 'postal_code': '36602',
 'timezone': 'America/Chicago',
 'has_upcoming_events': True,
 'id': 457220,
 'city': 'Mobile',
 'extended_address': 'Mobile, AL 36602',
 'display_location': 'Mobile, AL',
 'state': 'AL',
 'score': 0,
 'location': {'lat': 30.6897, 'lon': -88.0486},
 'access_method': None,
 'num_upcoming_events': 1,
 'address': '4 North Cedar Street',
 'capacity': 0,
 'slug': 'cedar-street-social-club',
 'name': 'Cedar Street Social Club',
 'url': 'https://seatgeek.com/venues/cedar-street-social-club/tickets',
 'country': 'US',
 'popularity': 0,
 'name_v2': 'Cedar Street Social Club'}

In [127]:
seatgeek['venue_name'] = seatgeek.venue.apply(lambda x: x['name'])
seatgeek['venue_popularity'] = seatgeek.venue.apply(lambda x: x['popularity'])
seatgeek['venue_address'] = seatgeek.venue.apply(lambda x: x['address'])
seatgeek['venue_city'] = seatgeek.venue.apply(lambda x: x['city'])
seatgeek['venue_state'] = seatgeek.venue.apply(lambda x: x['state'])
seatgeek['venue_zip'] = seatgeek.venue.apply(lambda x: x['postal_code'])
seatgeek['venue_id'] = seatgeek.venue.apply(lambda x: x['id'])
seatgeek.drop(columns=['venue'],inplace=True)

In [129]:
seatgeek.general_admission.fillna(False, inplace=True)

In [131]:
seatgeek.to_csv('sg_events.csv')